In [1]:
from __future__ import division
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from difflib import SequenceMatcher

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
#import knn classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#for validating your classification model
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dfb3=pd.read_csv('fda_DB_breastcancer.csv')
dfb3=dfb3.drop(['Unnamed: 0'], axis=1).reset_index(drop=True)
dfb3.head()

,drugname,age,gndr_cod,wt,reporter_country,de,lt,ho,ds,ca,ri,ot,pt,drugbank_ID,target,pathway,Class
0,zometa,74.0,0.0,44.0,japan,0.0,0.0,0.0,0.0,0.0,0.0,1.0,anaemia,DB00399,Farnesyl pyrophosphate synthase,Zoledronate Action Pathway,Bisphosphonates
1,zometa,74.0,0.0,44.0,japan,0.0,0.0,0.0,0.0,0.0,0.0,1.0,anaemia,DB00399,Geranylgeranyl pyrophosphate synthase,Zoledronate Action Pathway,Bisphosphonates
2,zometa,74.0,0.0,44.0,japan,0.0,0.0,0.0,0.0,0.0,0.0,1.0,anaemia,DB00399,Hydroxylapatite,Zoledronate Action Pathway,Bisphosphonates
3,zometa,74.0,0.0,44.0,japan,0.0,0.0,0.0,0.0,0.0,0.0,1.0,aortic valve incompetence,DB00399,Farnesyl pyrophosphate synthase,Zoledronate Action Pathway,Bisphosphonates
4,zometa,74.0,0.0,44.0,japan,0.0,0.0,0.0,0.0,0.0,0.0,1.0,aortic valve incompetence,DB00399,Geranylgeranyl pyrophosphate synthase,Zoledronate Action Pathway,Bisphosphonates


In [98]:
dfb3['target'].unique()

array(['Farnesyl pyrophosphate synthase',
       'Geranylgeranyl pyrophosphate synthase', 'Hydroxylapatite', nan,
       'Receptor tyrosine-protein kinase erbB-2',
       'Epidermal growth factor receptor', 'Complement C1r subcomponent',
       'Complement C1q subcomponent subunit A',
       'Complement C1q subcomponent subunit B',
       'Complement C1q subcomponent subunit C',
       'Complement C1s subcomponent',
       'High affinity immunoglobulin gamma Fc receptor I',
       'Low affinity immunoglobulin gamma Fc region receptor II-a',
       'Low affinity immunoglobulin gamma Fc region receptor II-b',
       'Low affinity immunoglobulin gamma Fc region receptor II-c',
       'Low affinity immunoglobulin gamma Fc region receptor III-B',
       'Low affinity immunoglobulin gamma Fc region receptor III-A',
       'Tubulin beta-1 chain', 'Apoptosis regulator Bcl-2',
       'Microtubule-associated protein 2',
       'Microtubule-associated protein 4',
       'Microtubule-associated pr

In [3]:
dummy_col = pd.get_dummies(dfb3[['drugname','reporter_country','target','pathway','Class']])
dfb4=dfb3.drop(['drugname','reporter_country','target','pathway','Class','pt','drugbank_ID'] , axis=1)
dfb4 = pd.concat([dfb4, dummy_col], axis=1)
dfb4.head()

,age,gndr_cod,wt,de,lt,ho,ds,ca,ri,ot,...,Class_Pyrimidines and pyrimidine derivatives,Class_Quaternary ammonium salts,Class_Quinone and hydroquinone lipids,Class_Sulfinylbenzimidazoles,Class_Transition metal chlorides,Class_Triazoles,Class_Tryptamines and derivatives,Class_Tyrosols and derivatives,Class_Vitamin D and derivatives,Class_Xylenes
0,74.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,74.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
2,74.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,74.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,74.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y=dfb4['de']
X=dfb4.drop(['de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=0)    #building 100 decision trees
clf1.fit(X_train, y_train)
print "Death model"
#print "oob score:", clf1.oob_score_
print metrics.accuracy_score(y_test, clf1.predict(X_test))

print metrics.confusion_matrix(y_test, clf1.predict(X_test))
TP=metrics.confusion_matrix(y_test, clf1.predict(X_test))[0,0]
TN=metrics.confusion_matrix(y_test, clf1.predict(X_test))[1,1]
FP=metrics.confusion_matrix(y_test, clf1.predict(X_test))[1,0]
FN=metrics.confusion_matrix(y_test, clf1.predict(X_test))[0,1]
print "Sensitivity=", TP/(TP+FN)
print "Specificity=", TN/(TN+FP)
#clf1.fit(X, y)
#scores = cross_val_score(clf1, X, y, scoring='accuracy', cv=10)
#print scores
#print scores.mean()
print "======================================================="
y=dfb4['lt']
X=dfb4.drop(['de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf2 = RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=0)    #building 100 decision trees
clf2.fit(X_train, y_train)
print "Life threating model"
#print "oob score:", clf2.oob_score_
print metrics.accuracy_score(y_test, clf2.predict(X_test))

print metrics.confusion_matrix(y_test, clf2.predict(X_test))
TP=metrics.confusion_matrix(y_test, clf2.predict(X_test))[0,0]
TN=metrics.confusion_matrix(y_test, clf2.predict(X_test))[1,1]
FP=metrics.confusion_matrix(y_test, clf2.predict(X_test))[1,0]
FN=metrics.confusion_matrix(y_test, clf2.predict(X_test))[0,1]
print "Sensitivity=", TP/(TP+FN)
print "Specificity=", TN/(TN+FP)
#clf2.fit(X, y)
#scores = cross_val_score(clf2, X, y, scoring='accuracy', cv=10)
#print scores
#print scores.mean()
print "======================================================="

y=dfb4['ho']
X=dfb4.drop(['de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf3 = RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=0)    #building 100 decision trees
clf3.fit(X_train, y_train)
print "Hospitalization model"
#print "oob score:", clf3.oob_score_
print metrics.accuracy_score(y_test, clf3.predict(X_test))

print metrics.confusion_matrix(y_test, clf3.predict(X_test))
TP=metrics.confusion_matrix(y_test, clf3.predict(X_test))[0,0]
TN=metrics.confusion_matrix(y_test, clf3.predict(X_test))[1,1]
FP=metrics.confusion_matrix(y_test, clf3.predict(X_test))[1,0]
FN=metrics.confusion_matrix(y_test, clf3.predict(X_test))[0,1]
print "Sensitivity=", TP/(TP+FN)
print "Specificity=", TN/(TN+FP)
#clf3.fit(X, y)
#scores = cross_val_score(clf3, X, y, scoring='accuracy', cv=10)
#print scores
#print scores.mean()
print "======================================================="

y=dfb4['ds']
X=dfb4.drop(['de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf4 = RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=0)    #building 100 decision trees
clf4.fit(X_train, y_train)
print "Disability model"
#print "oob score:", clf4.oob_score_
print metrics.accuracy_score(y_test, clf4.predict(X_test))

print metrics.confusion_matrix(y_test, clf4.predict(X_test))
TP=metrics.confusion_matrix(y_test, clf4.predict(X_test))[0,0]
TN=metrics.confusion_matrix(y_test, clf4.predict(X_test))[1,1]
FP=metrics.confusion_matrix(y_test, clf4.predict(X_test))[1,0]
FN=metrics.confusion_matrix(y_test, clf4.predict(X_test))[0,1]
print "Sensitivity=", TP/(TP+FN)
print "Specificity=", TN/(TN+FP)
#clf4.fit(X, y)
#scores = cross_val_score(clf4, X, y, scoring='accuracy', cv=10)
#print scores
#print scores.mean()
print "======================================================="

y=dfb4['ot']
X=dfb4.drop(['de','lt','ho','ds','ca','ri','ot'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf5 = RandomForestClassifier(n_estimators=50, class_weight='balanced', n_jobs=-1, random_state=0)    #building 100 decision trees
clf5.fit(X_train, y_train)
print "Other Serious issues model"
#print "oob score:", clf5.oob_score_
print metrics.accuracy_score(y_test, clf5.predict(X_test))

print metrics.confusion_matrix(y_test, clf5.predict(X_test))
TP=metrics.confusion_matrix(y_test, clf5.predict(X_test))[0,0]
TN=metrics.confusion_matrix(y_test, clf5.predict(X_test))[1,1]
FP=metrics.confusion_matrix(y_test, clf5.predict(X_test))[1,0]
FN=metrics.confusion_matrix(y_test, clf5.predict(X_test))[0,1]
print "Sensitivity=", TP/(TP+FN)
print "Specificity=", TN/(TN+FP)
#clf5.fit(X, y)
#scores = cross_val_score(clf5, X, y, scoring='accuracy', cv=10)
#print scores
#print scores.mean()
print "======================================================="

X_test=X_test.reset_index(drop=True)

Death model
0.9631260319207485
[[9442  325]
 [  77 1058]]
Sensitivity= 0.9667246851643289
Specificity= 0.93215859030837
Life threating model
0.9905521922583013
[[10380    35]
 [   68   419]]
Sensitivity= 0.9966394623139703
Specificity= 0.8603696098562629
Hospitalization model
0.9282700421940928
[[6871  490]
 [ 292 3249]]
Sensitivity= 0.9334329574786034
Specificity= 0.9175374188082462
Disability model
0.9943129700972299
[[10575    39]
 [   23   265]]
Sensitivity= 0.9963256076879593
Specificity= 0.9201388888888888
Other Serious issues model
0.9328563566318107
[[6893  487]
 [ 245 3277]]
Sensitivity= 0.9340108401084011
Specificity= 0.9304372515616127


In [92]:
age=20
wt=57
gndr=0
tr="Farnesyl pyrophosphate synthase"
cr='japan'
x=np.zeros(X_test.loc[0].shape)
X=pd.DataFrame(x)
out=pd.DataFrame(columns={'drug','pathway','score'})
li_dname=dfb3[dfb3['target'].str.contains(tr, na=False)]['drugname'].unique()
li_pname=dfb3[dfb3['target'].str.contains(tr, na=False)]['pathway'].dropna().unique()
li_tname=dfb3[dfb3['target'].str.contains(tr, na=False)]['target'].unique()
a=[]
for lid in li_dname:
    word_d="drugname_"+lid
    a.append(X_test.columns.get_loc(word_d))
b=[]
if len(li_pname) > 0:
    for lip in li_pname:
        word_p="pathway_"+lip
        b.append(X_test.columns.get_loc(word_p))
c=[]
for lit in li_tname:
    word_t="target_"+lit
    c.append(X_test.columns.get_loc(word_t))

In [93]:
out=pd.DataFrame(columns={'drug','pathway','score'})
x_drug=[]
for i in a:
    for j in b:
        for k in c:
            X=np.zeros(X_test.loc[0].shape)
            X[0]=age
            X[1]=wt
            X[2]=gndr
            X[X_test.columns.get_loc("reporter_country_"+cr)]=1
            X[i]=1
            X[j]=1
            X[k]=1
            x_drug.append(X.reshape(1, -1))
            out=out.append(pd.DataFrame({'drug':[X_test.columns[i]],'pathway':[X_test.columns[j]],'score':0}))
x_drugs=[f[0] for f in x_drug]
out=out.reset_index(drop=True)

In [94]:
w1=round(np.random.triangular(5,9,10))
w2=round(np.random.triangular(5,7,10))
w3=round(np.random.triangular(3,5,7))
w4=round(np.random.triangular(1,3,5))
w5=round(np.random.triangular(1,1,5))
val1=[f[1]*w1 for f in clf1.predict_proba(x_drugs)]
val2=[f[1]*w2 for f in clf2.predict_proba(x_drugs)]
val3=[f[1]*w3 for f in clf3.predict_proba(x_drugs)]
val4=[f[1]*w4 for f in clf4.predict_proba(x_drugs)]
val5=[f[1]*w5 for f in clf5.predict_proba(x_drugs)]
out['score']=[val1[i]+val2[i]+val3[i]+val4[i]+val5[i] for i in range(len(val1))]
out['score']=out['score']/sum(out['score'])
out=out.sort_values('score')
out['drug']=out['drug'].str.replace("drugname_", "")
out['pathway']=out['pathway'].str.replace("pathway_", "")
out2=out.groupby(['drug'])['score'].mean().reset_index()
out2=out2.sort_values('score').reset_index(drop=True)

In [96]:
def drug_recommender_pro(age,wt,gndr,cr,tr):
    x=np.zeros(X_test.loc[0].shape)
    X=pd.DataFrame(x)
    out=pd.DataFrame(columns={'drug','pathway','score'})
    li_dname=dfb3[dfb3['target'].str.contains(tr, na=False)]['drugname'].unique()
    li_pname=dfb3[dfb3['target'].str.contains(tr, na=False)]['pathway'].dropna().unique()
    li_tname=dfb3[dfb3['target'].str.contains(tr, na=False)]['target'].unique()
    a=[]
    for lid in li_dname:
        word_d="drugname_"+lid
        a.append(X_test.columns.get_loc(word_d))
    b=[]
    if len(li_pname) > 0:
        for lip in li_pname:
            word_p="pathway_"+lip
            b.append(X_test.columns.get_loc(word_p))
    c=[]
    for lit in li_tname:
        word_t="target_"+lit
        c.append(X_test.columns.get_loc(word_t))
    out=pd.DataFrame(columns={'drug','pathway','score'})
    x_drug=[]
    for i in a:
        for j in b:
            for k in c:
                X=np.zeros(X_test.loc[0].shape)
                X[0]=age
                X[1]=wt
                X[2]=gndr
                X[X_test.columns.get_loc("reporter_country_"+cr)]=1
                X[i]=1
                X[j]=1
                X[k]=1
                x_drug.append(X.reshape(1, -1))
                out=out.append(pd.DataFrame({'drug':[X_test.columns[i]],'pathway':[X_test.columns[j]],'score':0}))
    x_drugs=[f[0] for f in x_drug]
    out=out.reset_index(drop=True)
    w1=round(np.random.triangular(5,9,10))
    w2=round(np.random.triangular(5,7,10))
    w3=round(np.random.triangular(3,5,7))
    w4=round(np.random.triangular(1,3,5))
    w5=round(np.random.triangular(1,1,5))
    val1=[f[1]*w1 for f in clf1.predict_proba(x_drugs)]
    val2=[f[1]*w2 for f in clf2.predict_proba(x_drugs)]
    val3=[f[1]*w3 for f in clf3.predict_proba(x_drugs)]
    val4=[f[1]*w4 for f in clf4.predict_proba(x_drugs)]
    val5=[f[1]*w5 for f in clf5.predict_proba(x_drugs)]
    out['score']=[val1[i]+val2[i]+val3[i]+val4[i]+val5[i] for i in range(len(val1))]
    out['score']=out['score']/sum(out['score'])
    out=out.sort_values('score')
    out['drug']=out['drug'].str.replace("drugname_", "")
    out['pathway']=out['pathway'].str.replace("pathway_", "")
    out2=out.groupby(['drug'])['score'].mean().reset_index()
    out2=out2.sort_values('score').reset_index(drop=True)
    return(out2)

In [97]:
drug_recommender_pro(20,57,0,"japan","Farnesyl pyrophosphate synthase")

,drug,score
0,aredia,0.037607
1,zoledronic acid,0.038788
2,aclasta,0.038939
3,bondronat,0.040266
4,reclast,0.041857
5,ibandronic acid,0.043827
6,pamidronate disodium,0.044358
7,zometa,0.047692


In [102]:
tr=["Farnesyl pyrophosphate synthase", 'P-selectin','Hydroxylapatite','Geranylgeranyl pyrophosphate synthase','Epidermal growth factor receptor']

In [103]:
li_pname=dfb3[dfb3['target'].isin(tr)]['pathway'].dropna().unique()

In [104]:
li_pname

array(['Zoledronate Action Pathway', 'Pamidronate Action Pathway',
       'Trastuzumab Action Pathway', 'Erlotinib Action Pathway',
       'Gefitinib Action Pathway', 'Cetuximab Action Pathway',
       'Ibandronate Action Pathway', 'Panitumumab Action Pathway',
       'Lidocaine (Local Anaesthetic) Action Pathway',
       'Lidocaine (Antiarrhythmic) Action Pathway',
       'Lidocaine (Local Anaesthetic) Metabolism Pathway'], dtype=object)

In [129]:
dfb3['reporter_country'].unique()

array(['japan', 'france', 'canada', 'united states', 'mexico',
       'united kingdom', 'china', 'new zealand', 'germany', 'chile',
       'spain', 'india', 'poland', 'brazil', 'italy', 'portugal',
       'korea, republic of', 'south africa', 'hungary', 'singapore',
       'denmark', 'belgium', 'finland', 'country not specified',
       'croatia (local name: hrvatska)', 'netherlands', 'saudi arabia',
       'russian federation', 'ireland', 'sweden', 'austria', 'nicaragua',
       'hong kong', 'ghana', 'thailand', 'romania', 'lebanon',
       'switzerland', 'argentina', 'el salvador', 'czech republic',
       'turkey', 'australia', 'taiwan, province of china', 'greece',
       'norway', 'guatemala', 'peru', 'latvia', 'puerto rico',
       'philippines', 'colombia', 'viet nam', 'panama', 'cyprus',
       'israel', 'egypt', 'estonia', 'albania', 'serbia and montenegro',
       'malaysia', 'bulgaria', 'venezuela', 'slovakia (slovak republic)',
       'luxembourg', 'pakistan', 'ukraine', 'e

In [148]:
age=20
wt=57
gndr=0
tr=["Farnesyl pyrophosphate synthase", 'P-selectin','Hydroxylapatite','Geranylgeranyl pyrophosphate synthase','Epidermal growth factor receptor']
cr=['japan', 'united states']
x=np.zeros(X_test.loc[0].shape)
X=pd.DataFrame(x)
out=pd.DataFrame(columns={'drug','pathway','score'})
li_dname=dfb3[dfb3['target'].isin(tr)]['drugname'].unique()
li_pname=dfb3[dfb3['target'].isin(tr)]['pathway'].dropna().unique()
li_tname=dfb3[dfb3['target'].isin(tr)]['target'].unique()
li_cname=dfb3[dfb3['reporter_country'].isin(cr)]['reporter_country'].unique()
a=[]
for lid in li_dname:
    word_d="drugname_"+lid
    a.append(X_test.columns.get_loc(word_d))
b=[]
for lip in li_pname:
    word_p="pathway_"+lip
    b.append(X_test.columns.get_loc(word_p))
c=[]
for lit in li_tname:
    word_t="target_"+lit
    c.append(X_test.columns.get_loc(word_t))
d=[]
for lic in li_cname:
    word_c="reporter_country_"+lic
    d.append(X_test.columns.get_loc(word_c))

In [149]:
li_cname

array(['japan', 'united states'], dtype=object)

In [150]:
out=pd.DataFrame(columns={'drug','pathway','score'})
x_drug=[]
for i in a:
    for j in b:
        for k in c:
            for l in d:
                X=np.zeros(X_test.loc[0].shape)
                X[0]=age
                X[1]=wt
                X[2]=gndr
                #X[X_test.columns.get_loc("reporter_country_"+cr)]=1
                X[i]=1
                X[j]=1
                X[k]=1
                X[l]=1
                x_drug.append(X.reshape(1, -1))
                out=out.append(pd.DataFrame({'drug':[X_test.columns[i]],'pathway':[X_test.columns[j]],'score':0}))
x_drugs=[f[0] for f in x_drug]
out=out.reset_index(drop=True)

In [151]:

w1=round(np.random.triangular(5,9,10))
w2=round(np.random.triangular(5,7,10))
w3=round(np.random.triangular(3,5,7))
w4=round(np.random.triangular(1,3,5))
w5=round(np.random.triangular(1,1,5))
val1=[f[1]*w1 for f in clf1.predict_proba(x_drugs)]
val2=[f[1]*w2 for f in clf2.predict_proba(x_drugs)]
val3=[f[1]*w3 for f in clf3.predict_proba(x_drugs)]
val4=[f[1]*w4 for f in clf4.predict_proba(x_drugs)]
val5=[f[1]*w5 for f in clf5.predict_proba(x_drugs)]
out['score']=[val1[i]+val2[i]+val3[i]+val4[i]+val5[i] for i in range(len(val1))]
out['score']=out['score']/sum(out['score'])
out=out.sort_values('score').reset_index(drop=True)
out['drug']=out['drug'].str.replace("drugname_", "")
out['pathway']=out['pathway'].str.replace("pathway_", "")
out2=out.groupby(['drug'])['score'].mean().reset_index()
out2=out2.sort_values('score').reset_index(drop=True)

In [161]:
len([f[1] for f in clf1.predict_proba(x_drugs)])

2310

In [162]:
len(out)

2310

In [141]:
out2

,drug,score
0,erbitux,0.000338
1,hercepting,0.000389
2,aclasta,0.000390
3,zoledronic acid,0.000393
4,tarceva,0.000394
5,fragmin,0.000395
6,bondronat,0.000399
7,aredia,0.000400
8,reclast,0.000402
9,bonefos,0.000402


In [163]:
def drug_recommender_pro(age,wt,gndr,cr,tr):
    x=np.zeros(X_test.loc[0].shape)
    X=pd.DataFrame(x)
    out=pd.DataFrame(columns={'drug','pathway','score'})
    li_dname=dfb3[dfb3['target'].isin(tr)]['drugname'].unique()
    li_pname=dfb3[dfb3['target'].isin(tr)]['pathway'].dropna().unique()
    li_tname=dfb3[dfb3['target'].isin(tr)]['target'].unique()
    li_cname=dfb3[dfb3['reporter_country'].isin(cr)]['reporter_country'].unique()
    a=[]
    for lid in li_dname:
        word_d="drugname_"+lid
        a.append(X_test.columns.get_loc(word_d))
    b=[]
    if len(li_pname) > 0:
        for lip in li_pname:
            word_p="pathway_"+lip
            b.append(X_test.columns.get_loc(word_p))
    c=[]
    for lit in li_tname:
        word_t="target_"+lit
        c.append(X_test.columns.get_loc(word_t))
    d=[]
    for lic in li_cname:
        word_c="reporter_country_"+lic
        d.append(X_test.columns.get_loc(word_c))
    out=pd.DataFrame(columns={'drug','pathway','score'})
    x_drug=[]
    for i in a:
        for j in b:
            for k in c:
                for l in d:
                    X=np.zeros(X_test.loc[0].shape)
                    X[0]=age
                    X[1]=wt
                    X[2]=gndr
                    X[i]=1
                    X[j]=1
                    X[k]=1
                    X[l]=1
                    x_drug.append(X.reshape(1, -1))
                    out=out.append(pd.DataFrame({'drug':[X_test.columns[i]],'pathway':[X_test.columns[j]],'score':0}))
    x_drugs=[f[0] for f in x_drug]
    out=out.reset_index(drop=True)
    w1=round(np.random.triangular(5,9,10))
    w2=round(np.random.triangular(5,7,10))
    w3=round(np.random.triangular(3,5,7))
    w4=round(np.random.triangular(1,3,5))
    w5=round(np.random.triangular(1,1,5))
    val1=[f[1]*w1 for f in clf1.predict_proba(x_drugs)]
    val2=[f[1]*w2 for f in clf2.predict_proba(x_drugs)]
    val3=[f[1]*w3 for f in clf3.predict_proba(x_drugs)]
    val4=[f[1]*w4 for f in clf4.predict_proba(x_drugs)]
    val5=[f[1]*w5 for f in clf5.predict_proba(x_drugs)]
    out['score']=[val1[i]+val2[i]+val3[i]+val4[i]+val5[i] for i in range(len(val1))]
    out['score']=out['score']/sum(out['score'])
    out=out.sort_values('score').reset_index(drop=True)
    out['drug']=out['drug'].str.replace("drugname_", "")
    out['pathway']=out['pathway'].str.replace("pathway_", "")
    out2=out.groupby(['drug'])['score'].mean().reset_index()
    out2=out2.sort_values('score').reset_index(drop=True)
    return(out2)

In [164]:
drug_recommender_pro(20,57,0,["japan",'albania'],["Farnesyl pyrophosphate synthase",'DNA'])

,drug,score
0,pharmorubicin,0.000386
1,cytoxan,0.000403
2,xeloda,0.000413
3,gemcitabine,0.000433
4,doxorubicin hcl,0.000438
5,epirubicin hydrochloride,0.000439
6,doxorubicin,0.000440
7,cisplatin,0.000449
8,gemzar,0.000452
9,epirubicin,0.000458


In [147]:
c=["Japan",'Albania']
[i.lower() for i in c]

['japan', 'albania']